### How to read a file in blob storage and azure data lake store into pandas dataframe

** @author:Francis Kessie**

**install azure sdks using pip**
##pip install azure

### Make the connections to blob storage ##

In [17]:
# fill in your blob account name
blob_account_name = "name" 

# fill in your blob account key
blob_account_key = "key"  

# fill in the container name
mycontainer = "mycontainer"  

# fill in the blob name 
myblobname = "static-data/TestRefference.csv" 
# fill in the output file name, must create in source folder
mydatafile = "./out.csv"       

**Read file from blob storage into pandas dataframe **

In [18]:
import os 
import pandas as pd 
from azure.storage.blob import BlockBlobService
dirname = os.getcwd()
blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)

blob_service.get_blob_to_path(mycontainer, myblobname, mydatafile)

mydata = pd.read_csv(mydatafile, header = 0)

os.remove(os.path.join(dirname, mydatafile))

print(mydata.shape)

(6, 3)


In [26]:
#there you go with your dataframe
mydata

,Commodity,Tolerance,Tph_RecSetpoint
0,WHEAT,10,900
1,BARLEY,10,900
2,BARLEYMLT,10,600
3,OATS,10,750
4,LUPIN,10,600
5,CANOLA,10,750


### Reading a file from azure data lake store

** There are a number of ways to authenticate, this is one of them **

In [1]:
from azure.datalake.store import core, lib, multithread
token = lib.auth()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code HZLG3GJUA to authenticate.


In [73]:
#list files in data lake store
adls.ls('')

['catalog', 'plcreadings', 'system', 'thresholds']

In [71]:
#display content of particular files in the DLS
#adls.cat('/plcreadings/2017/10/19_0_8e3bcd635bba445e89a0b18a53f2a774.json')

In [15]:
adls.head('/plcreadings/2017/10/19_0_8e3bcd635bba445e89a0b18a53f2a774.json')

b'{"site":"Kwinana","trainno":"4K64","season":"2017","commodity":"BARLEY","messageid":6698,"iothub":{"MessageId":null,"CorrelationId":null,"ConnectionDeviceId":"kwinanapeaPLC","ConnectionDeviceGenerationId":"636426098977011561","EnqueuedTime":"2017-10-19T05:32:12.0720000Z","StreamId":null},"deviceid":"kwinanapeaPLC","grade":"BFOD1","b3_1_running_seconds":4966.0,"b3_1_feeding_seconds":3331.0,"b3_1_efficiency_pct":42.0,"b3_1_tph_setpoint":750.0,"c3_1_destination":"338","b3_1_actual_tph":539.0,"b3_1_tonnes_moved":458.0,"b3_2_running_seconds":455.0,"b3_2_feeding_seconds":326.0,"b3_2_efficiency_pct":42.0,"b3_2_tph_setpoint":750.0,"b3_2_destination":"102","b3_2_actual_tph":548.0,"b3_2_tonnes_moved":42.0,"b4_1_running_seconds":5020.0,"b4_1_feeding_seconds":3753.0,"b4_1_efficiency_pct":53.0,"b4_1_tph_setpoint":750.0,"b4_1_destination":"402","b4_1_actual_tph":584.0,"b4_1_tonnes_moved":588.0,"b4_2_running_second":5020.0,"b4_2_feeding_seconds":3753.0,"b4_2_efficiency_pct":52.0,"b4_2_tph_setpoint"

In [74]:
dlsURL = '/plcreadings/2017/10/19_0_a993aa71fc414a8cbcab366452b47f17.json'

In [75]:
with adls.open(dlsURL, blocksize=5**2000, delimiter=bytes) as fp:
    inp = fp.readlines()
    sd = [json.loads(inp[i]) for i in range(len(inp)) ]
    df3 = pd.DataFrame(sd)
df3.head()

,b3_1_actual_tph,b3_1_efficiency_pct,b3_1_feeding_seconds,b3_1_running_seconds,b3_1_tonnes_moved,b3_1_tph_setpoint,b3_2_actual_tph,b3_2_destination,b3_2_efficiency_pct,b3_2_feeding_seconds,...,commodity,deviceid,eventtimeawst,eventtimeutc,grade,iothub,messageid,season,site,trainno
0,1.0,68.0,3032.0,3458.0,590.0,850.0,0.0,430,65.0,3065.0,...,None,kwinanapeaPLC,2017-10-19T08:00:06.3044982Z,2017-10-19T00:00:06.3044982Z,None,"{'MessageId': None, 'CorrelationId': None, 'Co...",5667,None,Kwinana,4k14
1,0.0,68.0,3032.0,3468.0,590.0,850.0,1.0,430,65.0,3065.0,...,None,kwinanapeaPLC,2017-10-19T08:00:11.0389162Z,2017-10-19T00:00:11.0389162Z,None,"{'MessageId': None, 'CorrelationId': None, 'Co...",5668,None,Kwinana,4k14
2,0.0,68.0,3032.0,3473.0,590.0,850.0,1.0,430,65.0,3065.0,...,None,kwinanapeaPLC,2017-10-19T08:00:15.6365634Z,2017-10-19T00:00:15.6365634Z,None,"{'MessageId': None, 'CorrelationId': None, 'Co...",5669,None,Kwinana,4k14
3,0.0,68.0,3032.0,3478.0,590.0,850.0,2.0,430,65.0,3065.0,...,None,kwinanapeaPLC,2017-10-19T08:00:25.4335073Z,2017-10-19T00:00:25.4335073Z,None,"{'MessageId': None, 'CorrelationId': None, 'Co...",5670,None,Kwinana,4k14
4,2.0,68.0,3032.0,3488.0,590.0,850.0,0.0,430,65.0,3065.0,...,None,kwinanapeaPLC,2017-10-19T08:00:34.8586929Z,2017-10-19T00:00:34.8586929Z,None,"{'MessageId': None, 'CorrelationId': None, 'Co...",5672,None,Kwinana,4k14
